# Native APIs
Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate`
- `/get_server_args`
- `/get_model_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/get_memory_pool_size`
- `/update_weights`
- `/encode`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --port=30010
"""
)

wait_for_server("http://localhost:30010")

[2024-11-03 05:10:19] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, host='127.0.0.1', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=143631718, constrained_json_whitespace_pattern=None, decode_log_interval=40, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_init_addr=None

[2024-11-03 05:10:34 TP0] Init torch distributed begin.


[2024-11-03 05:10:35 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-03 05:10:35 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-03 05:10:35 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-03 05:10:35 weight_utils.py:288] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2024-11-03 05:10:36 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-11-03 05:10:43 TP0] max_total_num_tokens=2170757, max_prefill_tokens=16384, max_running_requests=4097, context_len=131072


[2024-11-03 05:10:43] INFO:     Started server process [1392516]
[2024-11-03 05:10:43] INFO:     Waiting for application startup.
[2024-11-03 05:10:43] INFO:     Application startup complete.
[2024-11-03 05:10:43] INFO:     Uvicorn running on http://127.0.0.1:30010 (Press CTRL+C to quit)


[2024-11-03 05:10:43] INFO:     127.0.0.1:45576 - "GET /v1/models HTTP/1.1" 200 OK


[2024-11-03 05:10:44] INFO:     127.0.0.1:45588 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-03 05:10:44 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:10:44] INFO:     127.0.0.1:45604 - "POST /generate HTTP/1.1" 200 OK
[2024-11-03 05:10:44] The server is fired up and ready to roll!


## Generate
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](../references/sampling_params.html).

In [2]:
import requests

url = "http://localhost:30010/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2024-11-03 05:10:48 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, cache hit rate: 6.67%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:10:49 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.75, #queue-req: 0
[2024-11-03 05:10:49 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 512.79, #queue-req: 0


[2024-11-03 05:10:49 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.00, gen throughput (token/s): 509.05, #queue-req: 0
[2024-11-03 05:10:49] INFO:     127.0.0.1:45022 - "POST /generate HTTP/1.1" 200 OK


## Get Server Args
Get the arguments of a server.

In [3]:
url = "http://localhost:30010/get_server_args"

response = requests.get(url)
print_highlight(response.json())

[2024-11-03 05:10:49] INFO:     127.0.0.1:45026 - "GET /get_server_args HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.

In [4]:
url = "http://localhost:30010/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json.keys() == {"model_path", "is_generation"}

[2024-11-03 05:10:49] INFO:     127.0.0.1:45028 - "GET /get_model_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = "http://localhost:30010/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2024-11-03 05:10:49 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 1, cache hit rate: 11.76%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:10:49] INFO:     127.0.0.1:45038 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = "http://localhost:30010/health"

response = requests.get(url)
print_highlight(response.text)

[2024-11-03 05:10:49] INFO:     127.0.0.1:45040 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = "http://localhost:30010/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2024-11-03 05:10:49] INFO:     127.0.0.1:45044 - "POST /flush_cache HTTP/1.1" 200 OK
[2024-11-03 05:10:49 TP0] Cache flushed successfully!


## Get Memory Pool Size

Get the memory pool size in number of tokens.


In [8]:
# get_memory_pool_size

url = "http://localhost:30010/get_memory_pool_size"

response = requests.get(url)
print_highlight(response.text)

[2024-11-03 05:10:49] INFO:     127.0.0.1:45052 - "GET /get_memory_pool_size HTTP/1.1" 200 OK


## Update Weights

Update model weights without restarting the server. Use for continuous evaluation during training. Only applicable for models with the same architecture and parameter size.

In [9]:
# successful update with same architecture and size

url = "http://localhost:30010/update_weights"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] == True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

[2024-11-03 05:10:49 TP0] Update weights begin. avail mem=4.97 GB


INFO 11-03 05:10:49 weight_utils.py:243] Using model weights format ['*.safetensors']


INFO 11-03 05:10:49 weight_utils.py:288] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.10it/s]

[2024-11-03 05:10:50 TP0] Update weights end.
[2024-11-03 05:10:50 TP0] Cache flushed successfully!
[2024-11-03 05:10:50] INFO:     127.0.0.1:45058 - "POST /update_weights HTTP/1.1" 200 OK


In [10]:
# failed update with different parameter size

url = "http://localhost:30010/update_weights"
data = {"model_path": "meta-llama/Llama-3.2-3B"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] == False
assert response_json["message"] == (
    "Failed to update weights: The size of tensor a (2048) must match "
    "the size of tensor b (3072) at non-singleton dimension 1.\n"
    "Rolling back to original weights."
)

[2024-11-03 05:10:50 TP0] Update weights begin. avail mem=4.97 GB


INFO 11-03 05:10:50 weight_utils.py:243] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-03 05:10:50 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-03 05:10:50 weight_utils.py:288] No model.safetensors.index.json found in remote.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]



Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.10it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.10it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]

[2024-11-03 05:10:51 TP0] Failed to update weights: The size of tensor a (2048) must match the size of tensor b (3072) at non-singleton dimension 1.
Rolling back to original weights.
[2024-11-03 05:10:51] INFO:     127.0.0.1:45066 - "POST /update_weights HTTP/1.1" 400 Bad Request


## Encode

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
terminate_process(server_process)

embedding_process = execute_shell_command(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --port 30020 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server("http://localhost:30020")

[2024-11-03 05:11:02] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, is_embedding=True, host='0.0.0.0', port=30020, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=226637862, constrained_json_whitespace_pattern=None, decode_log_interval=40, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_init_addr=None

[2024-11-03 05:11:17 TP0] Init torch distributed begin.


[2024-11-03 05:11:18 TP0] Load weight begin. avail mem=78.59 GB


[2024-11-03 05:11:18 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 11-03 05:11:18 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


...

[2024-11-03 05:11:33 TP0] max_total_num_tokens=1025173, max_prefill_tokens=16384, max_running_requests=4005, context_len=131072


[2024-11-03 05:11:33] INFO:     Started server process [1393352]
[2024-11-03 05:11:33] INFO:     Waiting for application startup.
[2024-11-03 05:11:33] INFO:     Application startup complete.
[2024-11-03 05:11:33] INFO:     Uvicorn running on http://0.0.0.0:30020 (Press CTRL+C to quit)


[2024-11-03 05:11:34] INFO:     127.0.0.1:39478 - "GET /v1/models HTTP/1.1" 200 OK


[2024-11-03 05:11:34] INFO:     127.0.0.1:39494 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-11-03 05:11:34 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-11-03 05:11:35] INFO:     127.0.0.1:39496 - "POST /encode HTTP/1.1" 200 OK
[2024-11-03 05:11:35] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = "http://localhost:30020/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2024-11-03 05:11:39 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-03 05:11:39] INFO:     127.0.0.1:54938 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)